In [1]:
# pip install opencv-python

In [2]:
# pip install mediapipe

In [3]:
import cv2
import numpy as np
import mediapipe as mp
import time
import math

In [4]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(False,1,1, 0.65,0.5)
mpDraw = mp.solutions.drawing_utils
tipIds = [4, 8, 12, 16, 20]

In [5]:
def findHands(img, draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
#     print(results.multi_hand_landmarks)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            if draw:
                mpDraw.draw_landmarks(img, handLms,
                mpHands.HAND_CONNECTIONS)

    return img

In [6]:
def findPosition(img, handNo=0, draw=True):
    xList = []
    yList = []
    bbox = []
    lmList = []
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
#     print(results)
    if results.multi_hand_landmarks:
        myHand = results.multi_hand_landmarks[handNo]
        for id, lm in enumerate(myHand.landmark):
            # print(id, lm)
            h, w, c = img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            xList.append(cx)
            yList.append(cy)
            # print(id, cx, cy)
            lmList.append([id, cx, cy])
            if draw:
                cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

    if len(xList) != 0:
        xmin, xmax = min(xList), max(xList)
    if len(yList) != 0:
        ymin, ymax = min(yList), max(yList)
        bbox = xmin, ymin, xmax, ymax

    if draw:
        cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
        (0, 255, 0), 2)
#     print(lmList)
    return lmList, bbox

In [7]:
def fingersUp(lmList):
    fingers = []
    # Thumb
    if lmList[0][tipIds[0]][1] > lmList[0][tipIds[0] - 1][1]:
        fingers.append(1)
    else:
        fingers.append(0)

    # Fingers
    for id in range(1, 5):
        if lmList[0][tipIds[id]][2] < lmList[0][tipIds[id] - 2][2]:
            fingers.append(1)
        else:
            fingers.append(0)

        # totalFingers = fingers.count(1)

    return fingers

In [8]:
def findDistance( p1, p2, img, draw=True,r=15, t=3):
    x1, y1 = lmList[p1][1:]
    x2, y2 = lmList[p2][1:]
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

    if draw:
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
        cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

    return length, img, [x1, y1, x2, y2, cx, cy]

In [9]:

#######################
brushThickness = 5
eraserThickness = 100
########################


# folderPath = "Header"
# myList = os.listdir(folderPath)
# print(myList)
# overlayList = []
# for imPath in myList:
#     image = cv2.imread(f'{folderPath}/{imPath}')
#     overlayList.append(image)
# print(len(overlayList))
# header = overlayList[0]
drawColor = (255, 0, 255)

cap = cv2.VideoCapture(1)
cap.set(3, 640)
cap.set(4, 480)

# detector = htm.handDetector(detectionCon=0.65,maxHands=1)
xp, yp = 0, 0
imgCanvas = np.zeros((480, 640, 3), np.uint8)
x_lef = None
y_lef = None
x_rig = None
y_rig = None
done = False
while True:

    # 1. Import image
    success, img = cap.read()
    # 2. Find Hand Landmarks
    img = findHands(img)
    lmList = findPosition(img, draw=False)
#     cv2.imshow("wow",img)
    
    if ((len(lmList[0]) != 0) and (len(lmList[1]) != 0)):

#         print(lmList[0][8][1:])

        # tip of index and middle fingers
        x1, y1 = lmList[0][8][1:]
        x2, y2 = lmList[0][12][1:]

        # 3. Check which fingers are up
        fingers = fingersUp( lmList)
#         print(fingers)
        # 4. If Selection Mode - Two finger are up
        if fingers[1]==True and fingers[2]==True:
            xp, yp = 0, 0
            if(x_lef != None and y_lef != None):
                x_rig = x1
                y_rig = y1
            print("Selection Mode")
            if(x_rig != None and y_rig != None):
                done = True
            # # Checking for the click
#             if y1 < 125:
#                 if 250 < x1 < 450:
# #                     header = overlayList[0]
#                     drawColor = (255, 0, 255)
#                 elif 550 < x1 < 750:
# #                     header = overlayList[1]
#                     drawColor = (255, 0, 0)
#                 elif 800 < x1 < 950:
# #                     header = overlayList[2]
#                     drawColor = (0, 255, 0)
#                 elif 1050 < x1 < 1200:
# #                     header = overlayList[3]
#                     drawColor = (0, 0, 0)
            cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)

        # 5. If Drawing Mode - Index finger is up
        if ((fingers[1]==True) and (fingers[2] == False) and (done == False)):
            cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
            print("Drawing Mode")
            if xp == 0 and yp == 0:
                xp, yp = x1, y1
            if x_lef == None:
                x_lef = xp
                y_lef = yp
            x_rig = x1
            y_rig = y1
#             cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
            img=cv2.rectangle(img,(x_lef,y_lef),(x_rig,y_rig),(255,0,0),2)
#             if drawColor == (0, 0, 0):
#                 cv2.line(img, (xp, yp), (x1, y1), drawColor, eraserThickness)
#                 cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
            
#             else:
#                 cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
#                 cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)

            xp, yp = x1, y1


        # # Clear Canvas when all fingers are up
        # if all (x >= 1 for x in fingers):
        #     imgCanvas = np.zeros((720, 1280, 3), np.uint8)
    print(x_lef,' ',x_rig)
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img,imgCanvas)


    # Setting the header image
#     img[0:125, 0:1280] = header
    # img = cv2.addWeighted(img,0.5,imgCanvas,0.5,0)
    cv2.imshow("Image", img)
#     cv2.imshow("Canvas", imgCanvas)
#     cv2.imshow("Inv", imgInv)
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()

None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection Mode
None   None
None   None
Selection Mode
None   None
Selection Mode
None   None
Selection 